<a href="https://colab.research.google.com/github/statistics-jun/2022-1-ESAA/blob/main/%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-a. MNIST 데이터를 훈련, 검증, 테스트 데이터로 나눈다.

In [34]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size = 0.8)

X_vaild, X_test, y_vaild, y_test = train_test_split(X_rem, y_rem, test_size = 0.5)

1-b. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시킨다. (n_estimators=100, random_state=42)

In [36]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
xrd_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

In [37]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(random_state=42)

In [38]:
from sklearn.neural_network import MLPClassifier
mlp_clf = MLPClassifier(random_state=42)

In [39]:
from sklearn.metrics import accuracy_score
for clf in (rnd_clf, xrd_clf, svm_clf, mlp_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.9536785714285714
ExtraTreesClassifier 0.9579285714285715


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC 0.8551428571428571
MLPClassifier 0.9211071428571429


1-c. 이들을 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [40]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators = [('rf', rnd_clf), ('xrf', xrd_clf), ('svm', svm_clf), ('mlp', mlp_clf)],
    voting='hard'
)

In [41]:
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
print(voting_clf.__class__.__name__, accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


VotingClassifier 0.9554642857142858


1-d. 다른 분류기보다 성능이 많이 떨어졌던 선형 SVM 제거 후 VotingClassifier를 다시 평가

In [42]:
voting_clf_new = VotingClassifier(
    estimators = [('rf', rnd_clf), ('xrf', xrd_clf), ('mlp', mlp_clf)],
    voting='hard'
)

In [43]:
voting_clf_new.fit(X_train, y_train)
y_pred = voting_clf_new.predict(X_test)
print(voting_clf_new.__class__.__name__, accuracy_score(y_test, y_pred))

VotingClassifier 0.9592142857142857


1-d. 간접 투표 분류기를 사용하여 앙상블로 연결.

In [44]:
voting_clf_soft = VotingClassifier(
    estimators = [('rf', rnd_clf), ('xrf', xrd_clf), ('mlp', mlp_clf)],
    voting='soft'
)

voting_clf_soft.fit(X_train, y_train)
y_pred = voting_clf_soft.predict(X_test)
print(voting_clf_soft.__class__.__name__, accuracy_score(y_test, y_pred))

VotingClassifier 0.9378571428571428
